In [1]:
import dspy
from data_models.dspy.dspy_o_function_summary import DspyFunctionSummary
from data_models.dspy.dspy_unoplat_fs_function_subset import DspyUnoplatFunctionSubset
from data_models.dspy.dspy_unoplat_fs_node_subset import DspyUnoplatNodeSubset

/Users/jghiya/Library/Caches/pypoetry/virtualenvs/unoplat-code-confluence-wBVFyKeX-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ollama_codestral = dspy.OllamaLocal(model='codestral:22b-v0.1-q8_0')
dspy.configure(lm=ollama_codestral)

In [3]:
class CodeConfluenceFunctionSignature(dspy.Signature):
    """This signature contains class metadata and function metadata with function content and returns descriptive function summary. Only respond in this json format"""
    dspy_class_subset: DspyUnoplatNodeSubset = dspy.InputField(desc="This will contain class metadata")
    dspy_function_subset: DspyUnoplatFunctionSubset = dspy.InputField(desc="This will contain function metadata with function content")
    dspy_function_summary: DspyFunctionSummary = dspy.OutputField(desc="This will contain function summary")



In [4]:
class CodeConfluenceFunctionSummaryOptimiserSignature(dspy.Signature):
    """This signature contains function summary and returns optimised and accurate function summary. Only respond in this json format"""
    dspy_function_input_summary: DspyFunctionSummary = dspy.InputField(desc="This will contain function summary")
    dspy_function_output_summary: DspyFunctionSummary = dspy.OutputField(desc="This will contain optimised function summary")


In [5]:
class CodeConfluenceFunctionModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_function_summary = dspy.Predict(CodeConfluenceFunctionSignature)
        self.optimise_function_summary = dspy.Predict(CodeConfluenceFunctionSummaryOptimiserSignature)

    def forward(self, function_metadata, class_metadata):
        function_summary = self.generate_function_summary( dspy_class_subset = class_metadata, dspy_function_subset= function_metadata)
        optimised_function_summary = self.optimise_function_summary(function_summary)
        print(optimised_function_summary.dspy_function_output_summary)
        return optimised_function_summary

In [6]:
dspy_pipeline = CodeConfluenceFunctionModule()

In [7]:
import json
from data_models.chapi_unoplat_node import Node
from data_models.dspy.dspy_unoplat_fs_annotation_subset import DspyUnoplatAnnotationSubset
from data_models.dspy.dspy_unoplat_fs_function_call_subset import DspyUnoplatFunctionCallSubset
from data_models.dspy.dspy_unoplat_fs_function_subset import DspyUnoplatFunctionSubset
from data_models.dspy.dspy_unoplat_fs_node_subset import DspyUnoplatNodeSubset

try:
    with open('springstarterjava1_codes.json', 'r') as file:
        springstarterjava1_codes = json.load(file)
except FileNotFoundError:
    print("Error: File 'springstarterjava1_codes.json' not found.")
    springstarterjava1_codes = []
except json.JSONDecodeError:
    print("Error: File 'springstarterjava1_codes.json' contains invalid JSON.")
    springstarterjava1_codes = []

node_subsets = []

function_subsets = []
count = 0
for item in springstarterjava1_codes:
    try:
        node = Node(**item)
        print("node name",node.node_name)
        node_subset = DspyUnoplatNodeSubset(
            NodeName=node.node_name,
            Imports=node.imports,
            Extend=node.extend,
            MultipleExtend=node.multiple_extend,
            Fields=node.fields,
            Annotations=[DspyUnoplatAnnotationSubset(Name=annotation.name,KeyValues=annotation.key_values) for annotation in node.annotations]
        )
        count = count + 1
        print(node_subset)
        for func in node.functions:
            print(func)
            function_subset = DspyUnoplatFunctionSubset(
                Name=func.name,
                ReturnType=func.return_type,
                Annotations=[DspyUnoplatAnnotationSubset(Name=annotation.name,KeyValues=annotation.key_values) for annotation in node.annotations],
                LocalVariables=func.local_variables,
                Content=func.content,
                FunctionCalls=[DspyUnoplatFunctionCallSubset(NodeName=call.node_name, FunctionName=call.function_name, Parameters=call.parameters) for call in func.function_calls]
            )
            pred = dspy_pipeline(class_metadata=node_subset,function_metadata=function_subset)
            print(pred.dspy_function_output_summary)
            exit()
            
        
    except AttributeError as e:
        print(f"Error processing node data: {e}")


node name Order
node_name='Order' multiple_extend=None fields=[ClassFieldModel(type_type='OrderPrimaryKey', type_key='key', annotations=[Annotation(name='PrimaryKey', key_values=[], position=Position(start_line=13, start_line_position=4, stop_line=13, stop_line_position=5))]), ClassFieldModel(type_type='Integer', type_key='productQuantity', annotations=[Annotation(name='Column', key_values=[ChapiUnoplatAnnotationKeyVal(key='"product_quantity"', value='"product_quantity"')], position=Position(start_line=16, start_line_position=4, stop_line=16, stop_line_position=30)), Annotation(name='CassandraType', key_values=[ChapiUnoplatAnnotationKeyVal(key='type', value='CassandraType.Name.INT')], position=Position(start_line=17, start_line_position=4, stop_line=17, stop_line_position=48))]), ClassFieldModel(type_type='String', type_key='productName', annotations=[Annotation(name='Column', key_values=[ChapiUnoplatAnnotationKeyVal(key='"product_name"', value='"product_name"')], position=Position(sta

/Users/jghiya/Library/Caches/pypoetry/virtualenvs/unoplat-code-confluence-wBVFyKeX-py3.12/lib/python3.12/site-packages/dsp/templates/template_v2.py:35: SyntaxWarning: invalid escape sequence '\s'
  match = re.search("(.*)(\s){(.*)}\s(.*\${.*})", template)
/Users/jghiya/Library/Caches/pypoetry/virtualenvs/unoplat-code-confluence-wBVFyKeX-py3.12/lib/python3.12/site-packages/dsp/templates/template_v2.py:42: SyntaxWarning: invalid escape sequence '\s'
  match = re.search("(.*)(\s){(.*)}", template)


AssertionError: Need format_handler for dspy_class_subset of type <class 'data_models.dspy.dspy_unoplat_fs_node_subset.DspyUnoplatNodeSubset'>

In [ ]:
node_subsets[0]

In [ ]:
import dspy
import os
from data_models.chapi_unoplat_function import Function
from data_models.chapi_unoplat_function_summary import FunctionSummary
from loader import JsonLoader
from loader import JsonParser

ollama_mistral = dspy.OllamaLocal(model='mistral:7b-instruct-fp16')
dspy.configure(lm=ollama_mistral)


In [ ]:
class UnoplatFunctionSummary(dspy.Signature):
    """Generate accurate Summary only based on user metadata about function"""
    
    input: Function = dspy.InputField(desc="will contain all relevant function metadata")
    output: FunctionSummary = dspy.OutputField(desc="summarisation of function")


In [ ]:
from typing import List
from data_models.chapi_unoplat_class_summary import ClassSummary
from data_models.chapi_unoplat_node import Node


class UnoplatClassSummary(dspy.Signature):
    """Generate accurate summary only based on List of Function summaries and class metadata"""
    input: List[FunctionSummary] = dspy.InputField(desc="This contains list of function summaries that needs to be understood in a step by step fashion.")
    context: Node = dspy.InputField(desc="This will contain class metadata." )
    output: ClassSummary = dspy.OutputField(desc="This should contain class summary based on user shared metadata.")

In [ ]:
from typing import List

class UnoplatModuleFunctionSummary(dspy.Module):
    """Generate summary for all functions of a class"""
    def __init__(self):
        super().__init__()
        self.generate_function_summary = dspy.TypedChainOfThought(UnoplatFunctionSummary)
        

    def forward(self, list_function = None) -> str:
        function_summaries_context = []
        
        for item in list_function:
            function_summaries_context.append(self.generate_function_summary(item))

        
        return function_summaries_context    


            
        

In [ ]:
from dspy.teleprompt import BootstrapFewShot
# Set up the teleprompter
teleprompter = BootstrapFewShot()

# Compile the program without a training dataset
compiled_summary = teleprompter.compile(UnoplatModuleClassSummary(),trainset=None)

#cot_predictor = dspy.TypedChainOfThought(UnoplatFunctionSummary)

In [ ]:
iload_json = JsonLoader()

iparse_json = JsonParser()


meadata_classes = iload_json.load_json_from_file("springstarterjava1_codes.json")

package_dict = iparse_json.parse_json_to_nodes(meadata_classes,None)

print(package_dict.keys)

function: Function = package_dict['com.datastax.examples.order'][2].functions[0]
print(function)

prediction = cot_predictor(input=function,options={"format": "json"})

ollama_mistral.inspect_history(n=1)

In [ ]:
print(prediction)